### 数据探索分析

In [119]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import json
import re
import copy
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='ticks')
plt.rcParams['font.family'] = ['Arial Unicode MS'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
sns.set_style('whitegrid',{'font.sans-serif':['Arial Unicode MS','Arial']}) # 样式设置

In [152]:
dt_file = '/Users/huangsir/Desktop/GEEX/模型/geex_credit_score/V01/train/data/dt/'
data = pd.read_excel(dt_file+'data.xlsx')
with open(dt_file+'features.txt','r') as f:
    features = f.read().split('\n')
print(len(features))
data.head(3)

595


,apply_date,id_number,mobile,br.ir_allmatch_days,br.ir_m12_cell_x_home_addr_cnt,br.ir_m12_id_x_home_addr_cnt,br.信用风险识别-线上现金分期,br.按手机号查询，距最早在银行机构申请的间隔天数,br.按手机号查询，距最早在非银机构申请的间隔天数,br.按手机号查询，距最近在银行机构申请的间隔天数,...,N_AGE,C_GENDER,ALI_SCORE,YM_SCORE,loan_amt,balance_capital,max_overdue,f6_max_overdue,curr_overdue,target
0,20200311,4370ba14897480c70b37a239dc14db52,a7da377985203f626a0fd050aa304203,24.0,1.0,1.0,478.0,182.0,208.0,24.0,...,23,女,36.84,743.0,3000.0,0.0,0.0,0.0,0.0,0
1,20200329,711b52b23ebb745e7e806ad619d21cdd,8858cf247bc5b462f408462494a9d766,1.0,0.0,0.0,559.0,207.0,347.0,13.0,...,33,男,56.20,831.0,5600.0,499.0,39.0,39.0,0.0,1
2,20200617,fe8d6ce9d8a3ccf198f0df6e6878eb95,27a1853ad9ce8ba1a9f3c21452d60f66,19.0,1.0,1.0,665.0,344.0,344.0,19.0,...,33,男,53.85,720.0,NaN,NaN,NaN,NaN,NaN,-999


In [153]:
print(data.shape)

(14910, 612)


In [154]:
#申请时间分布
data['apply_date'] = data['apply_date'].apply(lambda x:datetime.strptime(str(x),'%Y%m%d'))

In [155]:
import copy
dt = copy.deepcopy(data)
# 授信日期分布
dt['apply_mon'] = dt['apply_date'].apply(lambda x:x.month)
dt = dt.set_index('apply_date')
dt.groupby(by='apply_mon',dropna=False).count()['id_number']

apply_mon
3    5872
4    2955
5    3854
6    2229
Name: id_number, dtype: int64

In [124]:
# 支用月分布
dt['LOAN_DATE'].apply(lambda x:str(x)[2:5]).value_counts()

MAR    5418
MAY    3965
APR    3125
JUN    2175
JUL     225
JAN       1
NOV       1
Name: LOAN_DATE, dtype: int64

In [125]:
data.sample(2)

,apply_date,id_number,mobile,br.ir_allmatch_days,br.ir_m12_cell_x_home_addr_cnt,br.ir_m12_id_x_home_addr_cnt,br.信用风险识别-线上现金分期,br.按手机号查询，距最早在银行机构申请的间隔天数,br.按手机号查询，距最早在非银机构申请的间隔天数,br.按手机号查询，距最近在银行机构申请的间隔天数,...,N_AGE,C_GENDER,ALI_SCORE,YM_SCORE,loan_amt,balance_capital,max_overdue,f6_max_overdue,curr_overdue,target
12917,2020-03-24,acc77927489a852edfa1daf0f93e01ff,d59bbf48c60c03864c23be46667795ff,21.0,0.0,0.0,677.0,NaN,333.0,NaN,...,25,女,37.18,791.0,7500.0,670.0,0.0,0.0,0.0,0
1100,2020-03-07,c2d0ff3a84196b559e57f48d40d3a15b,d6e9dabe911a5e6e89f61901eb797c18,5.0,1.0,1.0,540.0,NaN,345.0,NaN,...,23,男,18.92,612.0,2000.0,0.0,0.0,0.0,0.0,0


In [126]:
num_cols = list(set(data.select_dtypes(include=['int','float']).columns)&set(features))
cat_cols = list(set(data.select_dtypes(include=['object','bool']).columns)&set(features))

In [95]:
# # 变量均值和中位数随着时间变化
# savefile = '/Users/huangsir/desktop/geex/模型/geex_credit_score/v01/train/model/eda_result/eda/特征分布/'
# # 补充
# # for col in ['OVERDUE_DAYS','N_AGE','ALI_SCORE','YM_SCORE']:
# for col in num_cols:
#     df = pd.DataFrame()
#     df[f'{col}.均值'] = dt[col].dropna().rolling(30).mean()
#     df[f'{col}.中值'] = dt[col].dropna().rolling(30).median()
#     plt.figure(figsize=(14,7))
#     sns.lineplot(data=df,linewidth=3,legend='auto',markers=False)
#     plt.savefig(savefile+f'{col}.png')
#     plt.close()
# print('finish')

In [127]:
# 性别分布
dt['C_GENDER'].value_counts()

女    8151
男    6759
Name: C_GENDER, dtype: int64

In [128]:
cat_cols[5:]

['zzc.D90.手机号码.机构类型列表',
 'zzc.D90.手机号码.贷款类型列表',
 'zzc.D720.手机号码.贷款类型列表',
 'zzc.D720.身份证号码.贷款类型列表',
 'zzc.D720.手机号码.机构类型列表',
 'zzc.D540.手机号码.贷款类型列表',
 'zzc.D360.身份证号码.贷款类型列表',
 'zzc.D360.手机号码.机构类型列表',
 'zzc.D30.身份证号码.机构类型列表',
 'zzc.D540.手机号码.机构类型列表',
 'zzc.D90.身份证号码.机构类型列表',
 'zzc.D720.身份证号码.机构类型列表',
 'zzc.D180.身份证号码.贷款类型列表',
 'zzc.D180.身份证号码.机构类型列表',
 'zzc.D540.身份证号码.贷款类型列表',
 'zzc.D360.手机号码.贷款类型列表',
 'zzc.D180.手机号码.机构类型列表',
 'C_GENDER']

In [129]:
all_insti = set()
for col in cat_cols:
    if col != 'C_GENDER':
        insti = set(data[col])
        for i in insti:
            if i is not np.nan:
                i = i.lower().replace('_','')
                j = set(str(i).split(','))
                all_insti = all_insti.union(j)

In [130]:
all_insti

{'autofinance',
 'balancetransfer',
 'carmortgage',
 'citybank',
 'consumerfinance',
 'creditcard',
 'guarantorcompany',
 'insurance',
 'iou',
 'leasingcompany',
 'microfinance',
 'microloan',
 'mlloan',
 'nationalbank',
 'netmicrofinance',
 'others',
 'ownerloan',
 'p2p',
 'paymentservice',
 'platform',
 'privatebank'}

In [131]:
insti_count = []
for col in cat_cols:
    if col != 'C_GENDER':
        insti = data[col].tolist()
        for i in insti:
            if i is not np.nan:
                i = i.lower().replace('_','')
                j = str(i).split(',')
                insti_count += j
Counter(insti_count)

Counter({'consumerfinance': 154138,
         'platform': 39865,
         'netmicrofinance': 36395,
         'privatebank': 63273,
         'microfinance': 53722,
         'others': 42033,
         'p2p': 38314,
         'citybank': 3102,
         'nationalbank': 6130,
         'autofinance': 1645,
         'leasingcompany': 1485,
         'iou': 1017,
         'insurance': 309,
         'guarantorcompany': 10,
         'paymentservice': 36,
         'creditcard': 73932,
         'microloan': 73757,
         'mlloan': 52422,
         'balancetransfer': 2730,
         'carmortgage': 1559,
         'ownerloan': 10})

In [132]:
# 特征衍生,one-hot
def getf(x:str,y:str):
    if x is not np.nan:
        x = x.lower().replace('_','')
        x = set(str(x).split(',')) # set
        if y in x:
            return 1
        else:
            return 0
    else:
        return 0

insti_features = []
for col in cat_cols:
    if col != 'C_GENDER':
        data[col+'_len'] = data[col].apply(lambda x:len(x) if x is not np.nan else 0)
        data[col+'_nan'] = data[col].apply(lambda x:1 if x is np.nan else 0)
        features.append(col+'_len')
        insti_features.append(col+'_nan')
        try:
            features.remove(col)
        except:
            pass
        for i in all_insti:
            var = '.'.join([col,i])
            value_list = [getf(x,i) for x in data[col].tolist()]
            cont1 = Counter(value_list).get(0,0)/len(data)
            cont2 = Counter(value_list).get(1,0)/len(data)
            if cont1 < 0.9 and cont2 < 0.9:
                insti_features.append(var)
                data[var] = value_list

In [133]:
# 性别映射
data['C_GENDER'] = data['C_GENDER'].apply(lambda x:1 if x=='男' else 2)

In [134]:
features_new = features + insti_features
data[features_new] = data[features_new]

In [135]:
data.sample(3)

,apply_date,id_number,mobile,br.ir_allmatch_days,br.ir_m12_cell_x_home_addr_cnt,br.ir_m12_id_x_home_addr_cnt,br.信用风险识别-线上现金分期,br.按手机号查询，距最早在银行机构申请的间隔天数,br.按手机号查询，距最早在非银机构申请的间隔天数,br.按手机号查询，距最近在银行机构申请的间隔天数,...,zzc.D360.手机号码.贷款类型列表.consumerfinance,zzc.D360.手机号码.贷款类型列表.mlloan,zzc.D180.手机号码.机构类型列表_len,zzc.D180.手机号码.机构类型列表_nan,zzc.D180.手机号码.机构类型列表.privatebank,zzc.D180.手机号码.机构类型列表.p2p,zzc.D180.手机号码.机构类型列表.others,zzc.D180.手机号码.机构类型列表.consumerfinance,zzc.D180.手机号码.机构类型列表.netmicrofinance,zzc.D180.手机号码.机构类型列表.microfinance
9651,2020-05-15,294c8003f49ca8d890ac6555db0f1ebf,b2c76e29a8ffe88e433bb867464664fe,26.0,0.0,0.0,593.0,247.0,354.0,91.0,...,0,1,38,0,0,0,1,1,1,0
6038,2020-03-09,faa30d0aaedb29097d67538126fa2112,4bca141e1cdea5f68ee7975e3aee6327,11.0,1.0,1.0,597.0,201.0,272.0,201.0,...,0,1,24,0,1,0,0,0,0,1
4019,2020-03-02,5af2ff00bcd7450c704aff6a7802b3a1,b938fb6c31f11372bd23051bda7d9c21,12.0,2.0,2.0,428.0,329.0,346.0,101.0,...,1,1,44,0,1,1,1,0,0,1


In [136]:
data['target_m2'].value_counts()

 0    12348
 1     1644
-1      918
Name: target_m2, dtype: int64

In [137]:
data['target'].value_counts()

 0      11480
 1       1561
-999     1223
 2        646
Name: target, dtype: int64

In [138]:
# 特征初筛
import toad
exclude = list(set(data.columns) - set(features_new))

data_new = data.loc[(data['target'] != -999)&(data['target'] != 2),:]

data_new,drop_cols = toad.selection.select(frame = data_new,target = data_new['target'],
                                           empty=0.6,
                                           iv=0.01,
                                           corr=0.8,
                                           return_drop=True,
                                           exclude = exclude)
print(data_new.shape)
data_new.sample(4)

(13041, 202)


,apply_date,id_number,mobile,br.ir_allmatch_days,br.ir_m12_cell_x_home_addr_cnt,br.信用风险识别-线上现金分期,br.按手机号查询，近12个月在银行机构申请最大间隔天数,br.按手机号查询，近12个月在银行机构申请最小间隔天数,br.按手机号查询，近12个月在银行机构申请机构数,br.按手机号查询，近12个月在非银机构-持牌消费金融机构申请机构数,...,zzc.D90.身份证号码.贷款类型列表_len,zzc.D90.手机号码.机构类型列表_len,zzc.D720.手机号码.机构类型列表_len,zzc.D720.手机号码.机构类型列表.netmicrofinance,zzc.D90.身份证号码.机构类型列表_nan,zzc.D180.身份证号码.贷款类型列表_len,zzc.D180.身份证号码.贷款类型列表.consumerfinance,zzc.D180.身份证号码.机构类型列表.netmicrofinance,zzc.D540.身份证号码.贷款类型列表_len,zzc.D360.手机号码.贷款类型列表_len
9782,2020-03-22,f4288ba4c17f1fd9cf94c1bdce22fe54,6514a773e9cd09ea97d7c2a0a24b0f41,7.0,0.0,530.0,NaN,NaN,1.0,3.0,...,16,0,16,0,0,27,1,1,27,0
5985,2020-03-22,7f5fe10be86772e8fa83587da90841f3,6cfb4db56ac04c0aaf5e74a5118af4f1,7.0,0.0,643.0,NaN,NaN,1.0,2.0,...,0,0,15,0,1,0,0,0,16,16
5513,2020-03-15,e74d9db30eb9ecbe429a3dce1a03b671,e6f5f193cce68a5ea9ef5e4f4b3395e5,22.0,0.0,658.0,176.0,2.0,3.0,4.0,...,16,0,0,0,0,16,1,0,16,0
2652,2020-05-08,308a1243d847c45cabe6cfc702f88d10,2f479e1a39671a59d341cc954304f9f6,1.0,0.0,492.0,91.0,0.0,2.0,7.0,...,0,0,63,1,1,0,0,0,54,42


In [139]:
features_new = list(set(features_new)& set(data_new.columns))
len(features_new)

163

In [141]:
features_new = features_new + ['mob.wd_score_v2']
features_new = list(set(features_new)-set(['td.zrf.大额分标准版_','td.zrf.银行分标准版_','mob.fraud_stage',
                                          'br.线下消费贷客群评分']))
len(features_new)

159

In [151]:
# # 删除众数率大于0.6的变量
md = ModeDeal(df = data_new,cols=features_new)
data_new, drop_cols = md.drop_mode(mode=0.5)
drop_cols

In [143]:
features_new = list(set(features_new) - set(drop_cols))
cat_feature_new = list(set(insti_features)&set(features_new))
len(features_new)

153

In [144]:
# 描述统计
report = DataReport(df=data_new,features=features_new,target='target')
report_sum = report.reportSummary.sort_values(by='iv',axis=0,ascending=False,ignore_index=True) # 报告概述
report_det = report.reportDetail.sort_values(by='total_iv',axis=0,ascending=False,ignore_index=True) # 报告详情

In [145]:
report_sum.to_excel('./eda_result/eda/预筛特征报告概述.xlsx',index=False)
report_det.to_excel('./eda_result/eda/预筛特征报告详情.xlsx',index=False)

In [146]:
label_cols = [i for i in exclude if '列表' not in i]
label_cols

['balance_capital',
 'FINANCE_PRODUCT_CODE',
 'name',
 'curr_overdue',
 'id_number',
 'OVERDUE_DAYS',
 'apply_date',
 'max_overdue',
 'LOAN_DATE',
 'target',
 'f6_max_overdue',
 'APP_CHANNEL',
 'C_APP_ID',
 'mobile',
 'loan_amt',
 'target_m2',
 'TENOR']

In [147]:
# 保存预筛和数据
data_new = data[label_cols+features_new]
data_new.to_excel(dt_file+'data_new.xlsx',index=False)

with open(dt_file+'feature_new.txt','w') as f:
    f.write('\n'.join(features_new))
    
with open(dt_file+'cat_feature_new.txt','w') as f:
    f.write('\n'.join(cat_feature_new))

In [148]:
# 保留全量数据
data.to_excel(dt_file+'data_all.xlsx',index=False)

In [116]:
class BaseDes(object):

    def __get_top(self, series: pd.Series, top: int = 3, reverse: bool = False):
        ity = 'top'
        counts = series.value_counts()
        counts = list(zip(counts.index, counts, counts.divide(series.size)))
        if reverse:
            counts.reverse()
            ity = 'bottom'
        template = '{0[0]}:{0[2]:.1%}'
        index = [ity + str(i + 1) for i in range(top)]
        values = [template.format(counts[i]) if i < len(counts) else None for i in range(top)]
        return pd.Series(values, index=index)

    def __get_des(self, series: pd.Series, percentiles: list = None):
        if not percentiles:
            percentiles = [0.25, 0.5, 0.75]
        d = series.describe(percentiles)
        return d.drop('count')

    def __count_blank(self, series: pd.Series):
        n = series.isnull().sum()
        return '{0:.1%}'.format(n / series.size)

    def __is_numeric(self, series: pd.Series):
        return series.dtype.kind in 'ifc'

    def get_base_des(self, df: pd.DataFrame):
        rows = []
        for name, series in df.items():
            columns = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
            if self.__is_numeric(series):
                desc = self.__get_des(series)
                details = desc.tolist()
            else:
                top3 = self.__get_top(series, top=3)
                bottom4 = self.__get_top(series, top=4, reverse=True)
                details = top3.tolist() + bottom4[::-1].tolist()

            p_blank = self.__count_blank(series)

            row = pd.Series(
                index=['type', 'size', 'missing', 'unique'] + columns,
                data=[series.dtype, series.size, p_blank, series.nunique()] + details
            )

            row.name = name
            rows.append(row)

        return pd.DataFrame(rows)

In [117]:
import statsmodels.api as sm

def str2num(s):
    if '.' in str(s):
        try:
            return float(s)
        except:
            return s
    else:
        try:
            return int(float(s))
        except:
            return s
        
class VariableTrend(object):

    def __p_value_square(self, x, y):
        x = np.column_stack((x, x ** 2))
        x = sm.add_constant(x)
        lm = sm.OLS(y, x).fit()
        p = lm.pvalues  # 平方项的p值为第三项
        params = lm.params
        return (p, params)  # params为系数，平方项为第三项

    def __p_value(self, x, y):
        x = sm.add_constant(x)
        lm = sm.OLS(y, x).fit()
        p = lm.pvalues  # 此处设法取出p值并返回
        params = lm.params
        return (p, params)  # params为系数

    def bad_trend(self, x: np.array, y: np.array):
        """
        变量趋势
        :param x:0,1,2,3
        :param y:bad_rate
        """
        p2, params_square = self.__p_value_square(x, y)
        p1, params = self.__p_value(x, y)
        if len(p2) > 2:
            if p2[2] < 0.5 or p2[2] is np.nan:
                if params_square[2] > 0:
                    return ('上U')
                else:
                    return ('下U')
            elif p1[1] < 0.5:
                if params[1] > 0:
                    return ('单调递增')
                else:
                    return ('单调递减')
            else:
                return ('不明显')
        else:
            return ('无法计算')


class DataReport(VariableTrend, BaseDes):
    def __init__(self, df: pd.DataFrame, features: list, target: str, **kwargs):
        super().__init__(**kwargs)
        self.dt = df
        self.features = features
        self.target = target
        self.cols = ['variable', 'bin', 'total', 'bad', 'bad_rate','total_iv']
        self.reportDetail = self.report_detail(**kwargs)
        self.reportSummary = self.report_summary()

    def report_detail(self, q=10):
        """
        变量趋势分析
        """
        import copy
        data = copy.deepcopy(self.dt)
        report_detail_df = pd.DataFrame()
        for col in self.features:
            try:
                data[col] = data[col].apply(lambda x:str2num(x))
                _, bins = pd.qcut(data[col], q=q, retbins=True, duplicates='drop')
                bins = list(bins)
                del bins[-1]
                bins.insert(0,float('-inf'))
                bins.append(float('inf'))
            except TypeError as e:
                warnings.warn(f'变量趋势分析:{col},{e}')
                bins = None
            if bins and len(bins) > 3:
                total_cut = pd.cut(x=data[col], bins=bins, right=True)
                bad_cut = pd.cut(x=data.loc[data[self.target] == 1, col], bins=bins, right=True)
                good_cut = pd.cut(x=data.loc[data[self.target] == 0, col], bins=bins, right=True)
            else:
                total_cut = data[col]
                bad_cut = data.loc[data[self.target] == 1, col]
                good_cut = data.loc[data[self.target] == 0, col]

            # total
            total_df = pd.value_counts(total_cut, sort=False, dropna=False)
            total_sum = len(total_cut) if  not total_cut.empty else 0.001 # 总计
            # good
            good_df = pd.value_counts(good_cut, sort=False, dropna=False)  # 每个分数段好客户计数
            good_sum = len(good_cut) if not good_cut.empty else 0.001 # 好客户总数
            # bad
            bad_df = pd.value_counts(bad_cut, sort=False, dropna=False)  # 每个分数段的坏客户计数
            bad_sum = len(bad_cut) if not bad_cut.empty else 0.001 # 坏客户总数

            # 总计,好,坏 计数合并
            df = pd.concat([total_df, good_df, bad_df], axis=1)
            df.columns = ['total', 'good', 'bad']
            # 累积
            df_cumsum = df.cumsum()
            df_cumsum.columns = ['sum_total', 'sum_good', 'sum_bad']  # 累积总,累积好,累积坏 计数
            df = pd.concat([df, df_cumsum], axis=1)
            df['sum_total_rate'] = df['sum_total'] / total_sum  # 累积坏比率
            df['sum_good_rate'] = df['sum_good'] / good_sum  # 累积好比率
            df['sum_bad_rate'] = df['sum_bad'] / bad_sum  # 累积坏比率
            df['bad_rate'] = df['bad'] / df['total']  # 每个分数段的坏比率
            df['good_rate'] = df['good'] / df['total']  # 每个分数段的好比率
            df['lift'] = df['sum_bad_rate'] / df['sum_total_rate']
            df['woe'] = np.log((df['good'] / good_sum) / (df['bad'] / bad_sum))
            df['bin_iv'] = (df['good'] / good_sum - df['bad'] / bad_sum) * df['woe']
            df['total_iv'] = df.loc[(df['bin_iv'] != float('inf')) & (df['bin_iv'] > 0), 'bin_iv'].sum()
            df.reset_index(drop=False, inplace=True)
            df.rename(columns={'index': 'bin'}, inplace=True)
            df['variable'] = col
            df = df[self.cols]
            report_detail_df = pd.concat([report_detail_df, df])
        return report_detail_df

    def report_summary(self):
        """
        变量趋势摘要
        """
        data = copy.deepcopy(self.dt)
        rep = self.get_base_des(self.dt[self.features])
        rep = rep.reset_index().rename(columns={'index': 'variable'})
        iv, ks, trend, mode = [], [], [], []
        for col in rep['variable'].tolist():
            iv_tmp = self.reportDetail.loc[self.reportDetail['variable'] == col, 'total_iv'].unique()[0]
            iv.append(iv_tmp)
            try:
                data[col].fillna(-9999, inplace=True)
                mv = data[col].mode()[0]  # 众数
                mr = len(data.loc[data[col] == mv, :]) / len(data)  # 众数率
            except:
                mr = 1
            mode.append(mr)
            # trend
            y = self.reportDetail.loc[(self.reportDetail['variable'] == col) & (self.reportDetail['bin'].notnull()), 'bad_rate']
            x = range(len(y))
            if len(y) > 1:
                trd = self.bad_trend(x=np.array(x), y=np.array(y))
            else:
                trd = '无趋势'
                warnings.warn(f"变量 {col} 无趋势########################")
            trend.append(trd)
        rep['iv'] = iv
        rep['trend'] = trend
        rep['mode'] = mode
        return rep

In [118]:
class ModeDeal(object):
    """集中度处理"""
    def __init__(self,df:pd.DataFrame,cols:list=None):
        self.df = copy.deepcopy(df)
        if not cols:
            self.cols = df.columns.tolist()
        else:
            self.cols = copy.deepcopy(cols)
        self.mode_values = self.mode_count()

    def mode_count(self):
        mode_values = self.df[self.cols].mode(dropna=True).iloc[0].to_dict()
        mode_ratio = {k:Counter(self.df[k]).get(v)/len(self.df)
                      for k,v in mode_values.items()}
        return mode_ratio

    def drop_mode(self,keep_cols:list=None,return_drop:bool=True,mode:float=0.8):
        if not keep_cols:
            keep_cols = []
        drop_cols = [k for k in self.cols if self.mode_values.get(k) >= mode and k not in keep_cols]
        self.df = self.df[self.df.columns.difference(drop_cols)]
        if return_drop:
            return self.df, drop_cols
        else:
            return self.df